# PyPSA: Introduction

<img src="https://docs.pypsa.org/latest/assets/logo/logo-primary-light.svg#only-light" width="300px" />

:::{dropdown} YouTube Video
:::{iframe} https://www.youtube.com/embed/ORZgewOo1Ro
:width: 100%
Video not loading? Click [here](https://www.youtube.com/watch?v=ORZgewOo1Ro).
:::

PyPSA stands for **Python for Power System Analysis**.

PyPSA is an open source Python package for simulating and optimising modern energy systems that include features such as

- conventional generators with unit commitment (ramp-up, ramp-down, start-up, shut-down),
- time-varying wind and solar generation,
- energy storage with efficiency losses and inflow/spillage for hydroelectricity
- coupling to other energy sectors (electricity, transport, heat, industry),
- conversion between energy carriers (e.g. electricity to hydrogen),
- transmission networks (AC, DC, other fuels)

PyPSA can be used for a variety of problem types (e.g. electricity market modelling, long-term investment planning, transmission network expansion planning), and is designed to scale well with large networks and long time series.

Compared to building power system by hand in `linopy`, PyPSA does the following things for you:

- manage data inputs in standardised format
- build standardised optimisation problem
- communicate with the solver(s)
- retrieve and process optimisation results
- manage data outputs in standardised format

The [documentation](https://docs.pypsa.org/) describes it as follows: 

> PyPSA is an open-source Python framework for optimising and simulating modern power and energy systems that include features such as conventional generators with unit commitment, variable wind and solar generation, hydro-electricity, inter-temporal storage, coupling to other energy sectors, elastic demands, and linearised power flow with loss approximations in DC and AC networks. PyPSA is designed to scale well with large networks and long time series. It is made for researchers, planners and utilities with basic coding aptitude who need a fast, easy-to-use and transparent tool for power and energy system analysis.

### Key Dependencies

- `pandas` for storing data about network components and time series
- `numpy` and `scipy` for linear algebra and sparse matrix calculations
- `matplotlib` and `cartopy` for plotting on a map
- `networkx` for network calculations
- `linopy` for handling optimisation problems

:::{note}
Documentation for this package is available at https://docs.pypsa.org.
:::

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Download the `.ipynb` file using the download button on the top right corner and import it in [Google Colab](https://colab.research.google.com/).

Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install pypsa matplotlib cartopy highspy
```
:::

## Basic Structure

| Component | Description |
| --- | --- |
| [Network](https://docs.pypsa.org/latest/user-guide/design/#network-components) | Container for all components. |
| [Bus](https://docs.pypsa.org/latest/user-guide/components/buses/) | Node where components attach. |
| [Carrier](https://docs.pypsa.org/latest/user-guide/components/carriers/) | Energy carrier or technology (e.g. electricity, hydrogen, gas, coal, oil, biomass, on-/offshore wind, solar). Can track properties such as specific carbon dioxide emissions or nice names and colors for plots. |
| [Load](https://docs.pypsa.org/latest/user-guide/components/loads/) | Energy consumer (e.g. electricity demand). |
| [Generator](https://docs.pypsa.org/latest/user-guide/components/generators/) | Generator (e.g. power plant, wind turbine, PV panel). |
| [Line](https://docs.pypsa.org/latest/user-guide/components/lines/) | Power distribution and transmission lines (overhead and cables). |
| [Link](https://docs.pypsa.org/latest/user-guide/components/links/) | Links connect two buses with controllable energy flow, direction-control and losses. They can be used to model: <ul><li>HVDC links</li><li>HVAC li'nes (neglecting KVL, only net transfer capacities (NTCs))</li><li>conversion between carriers (e.g. electricity to hydrogen in electrolysis)</li></ul> |
| [StorageUnit](https://docs.pypsa.org/latest/user-guide/components/storage-units/) | Storage with fixed nominal energy-to-power ratio. |
| [GlobalConstraint](https://docs.pypsa.org/latest/user-guide/components/global-constraints/) | Constraints affecting many components at once, such as emission limits. |
| [Store](https://docs.pypsa.org/latest/user-guide/components/stores/) | Storage with separately extendable energy capacity. |


:::{note}
Links in the table lead to documentation for each component.
:::

<img src="https://docs.pypsa.org/latest/assets/images/buses.png" width="500px" />


:::{warning}
Per unit values of voltage and impedance are used internally for network calculations. It is assumed internally that the base power is **1 MW**.
:::

## From structured data to optimisation

**The design principle of PyPSA is that each component is associated with a set of variables and constraints that will be added to the optimisation model based on the input data stored for the components.**

For an *hourly* electricity market simulation, PyPSA will solve an optimisation problem that looks like this

\begin{equation}
\min_{g_{i,s,t}; f_{\ell,t}; g_{i,r,t,\text{charge}}; g_{i,r,t,\text{discharge}}; e_{i,r,t}} \sum_s o_{s} g_{i,s,t}
\end{equation}
such that
\begin{align}
0 & \leq g_{i,s,t} \leq \hat{g}_{i,s,t} G_{i,s}  & \text{generation limits : generator} \\
-F_\ell &\leq f_{\ell,t} \leq F_\ell & \text{transmission limits : line}  \\
d_{i,t} &= \sum_s g_{i,s,t} + \sum_r g_{i,r,t,\text{discharge}} - \sum_r g_{i,r,t,\text{charge}} - \sum_\ell K_{i\ell} f_{\ell,t} & \text{KCL : bus} \\
 0 &=\sum_\ell C_{\ell c} x_\ell f_{\ell,t} & \text{KVL : cycles} \\
0 & \leq g_{i,r,t,\text{discharge}} \leq G_{i,r,\text{discharge}}& \text{discharge limits : storage unit} \\
    0 & \leq g_{i,r,t,\text{charge}} \leq G_{i,r,\text{charge}} & \text{charge limits : storage unit} \\
    0 & \leq e_{i,r,t} \leq E_{i,r} & \text{energy limits : storage unit} \\ 
    e_{i,r,t} &= \eta^0_{i,r,t} e_{i,r,t-1} + \eta^1_{i,r,t}g_{i,r,t,\text{charge}} -  \frac{1}{\eta^2_{i,r,t}} g_{i,r,t,\text{discharge}} & \text{consistency : storage unit} \\
    e_{i,r,0} & = e_{i,r,|T|-1}  & \text{cyclicity : storage unit}
\end{align}

**Decision variables:**

- $g_{i,s,t}$ is the generator dispatch at bus $i$, technology $s$, time step $t$,
- $f_{\ell,t}$ is the power flow in line $\ell$,
- $g_{i,r,t,\text{dis-/charge}}$ denotes the charge and discharge of storage unit $r$ at bus $i$ and time step $t$,
- $e_{i,r,t}$ is the state of charge of storage $r$ at bus $i$ and time step $t$.

**Parameters:**

- $o_{i,s}$ is the marginal generation cost of technology $s$ at bus $i$,
- $x_\ell$ is the reactance of transmission line $\ell$,
- $K_{i\ell}$ is the incidence matrix,
- $C_{\ell c}$ is the cycle matrix,
- $G_{i,s}$ is the nominal capacity of the generator of technology $s$ at bus $i$,
- $F_{\ell}$ is the rating of the transmission line $\ell$,
- $E_{i,r}$ is the energy capacity of storage $r$ at bus $i$,
- $\eta^{0/1/2}_{i,r,t}$ denote the standing (0), charging (1), and discharging (2) efficiencies.

:::{note}
For a full reference to the optimisation problem description, see https://docs.pypsa.org/latest/user-guide/optimization/overview/
:::

## Simple electricity market example

Let's get acquainted with PyPSA to build a variant of one of the simple electricity market models we previously built in `linopy`.

We have the following data:

- fuel costs in € / MWh$_{th}$ 

In [ ]:
fuel_cost = dict(
    coal=8,
    gas=100,
    oil=48,
)

- efficiencies of thermal power plants in MWh$_{el}$ / MWh$_{th}$

In [ ]:
efficiency = dict(
    coal=0.33,
    gas=0.58,
    oil=0.35,
)

- specific emissions in t$_{CO_2}$ / MWh$_{th}$

In [ ]:
# t/MWh thermal
emissions = dict(
    coal=0.34,
    gas=0.2,
    oil=0.26,
    hydro=0,
    wind=0,
)

- power plant capacities in MW

In [ ]:
power_plants = {
    "SA": {"coal": 35000, "wind": 3000, "gas": 8000, "oil": 2000},
    "MZ": {"hydro": 1200},
}

- electrical load in MW

In [ ]:
loads = {
    "SA": 42000,
    "MZ": 650,
}

## Building a basic network

By convention, PyPSA is imported without an alias:

In [ ]:
import pypsa

First, we create a new network object which serves as the overall container for all components.

In [ ]:
n = pypsa.Network()

The second component we need are buses. **Buses** are the fundamental nodes of the network, to which all other components like loads, generators and transmission lines attach. They enforce energy conservation for all elements feeding in and out of it (i.e. Kirchhoff’s Current Law).

<img src="https://docs.pypsa.org/latest/assets/images/buses.png" width="500px" />

Components can be added to the network `n` using the `n.add()` function. It takes the component name as a first argument, the name of the component as a second argument and possibly further parameters as keyword arguments. Let's use this function, to add buses for each country to our network:

In [ ]:
n.add("Bus", "SA", y=-30.5, x=25, v_nom=400)
n.add("Bus", "MZ", y=-18.5, x=35.5, v_nom=400)

For each class of components, the data describing the components is stored in a `pandas.DataFrame`. For example, all static data for buses is stored in `n.buses`

In [ ]:
n.buses

You see there are many more attributes than we specified while adding the buses; many of them are filled with default parameters which were added. You can look up the field description, defaults and status (required input, optional input, output) for buses here https://docs.pypsa.org/latest/user-guide/components/buses/, and analogously for all other components. 

The method `n.add()` also allows you to add multiple components at once. For instance, multiple **carriers** for the fuels with information on specific carbon dioxide emissions, a nice name, and colors for plotting. For this, the function takes the component name as the first argument and then a list of component names and then optional arguments for the parameters. Here, scalar values, lists, dictionary or `pandas.Series` are allowed. The latter two needs keys or indices with the component names.

In [ ]:
n.add(
    "Carrier",
    ["coal", "gas", "oil", "hydro", "wind"],
    co2_emissions=emissions,
    nice_name=["Coal", "Gas", "Oil", "Hydro", "Onshore Wind"],
    color=["dimgrey", "tomato", "olive", "seagreen", "royalblue"],
)

n.add("Carrier", "AC", nice_name="Electricity", color="crimson")

The `n.add()` function is very general. It lets you add any component to the network object `n`. For instance, in the next step we add **generators** for all the different power plants.

In Mozambique:

In [ ]:
n.add(
    "Generator",
    "MZ hydro",
    bus="MZ",
    carrier="hydro",
    p_nom=1200,  # MW
    marginal_cost=0,  # default
)

In South Africa (in a loop):

In [ ]:
for tech, p_nom in power_plants["SA"].items():
    n.add(
        "Generator",
        f"SA {tech}",
        bus="SA",
        carrier=tech,
        efficiency=efficiency.get(tech, 1),
        p_nom=p_nom,
        marginal_cost=fuel_cost.get(tech, 0) / efficiency.get(tech, 1),
    )

As a result, the `n.generators` DataFrame looks like this:

In [ ]:
n.generators

Next, we're going to add the electricity demand. This time, without a loop.

A positive value for `p_set` means consumption of power from the bus (in MW).

In [ ]:
n.add(
    "Load",
    "SA electricity demand",
    bus="SA",
    p_set=loads["SA"],
    carrier="AC",
)

In [ ]:
n.add(
    "Load",
    "MZ electricity demand",
    bus="MZ",
    p_set=loads["MZ"],
    carrier="AC",
)

In [ ]:
n.loads

Finally, we add the connection between Mozambique and South Africa with a 500 MW line:

In [ ]:
n.add(
    "Line",
    "SA-MZ",
    bus0="SA",
    bus1="MZ",
    s_nom=500,
    x=1,
    r=1,
)

In [ ]:
n.lines

We can have a sneak peek at the network we built with the `n.plot()` function.

In [ ]:
n.plot(bus_size=1, margin=1);

Or with its interactive sibling `n.explore()`.

In [ ]:
n.explore(bus_size=2e3, line_width=10)

## Optimisation

With all input data transferred into PyPSA's data structure, we can now build and run the resulting optimisation problem. In PyPSA, building, solving and retrieving results from the optimisation model is contained in a single function call `n.optimize()`. This function optimizes the operational dispatch decisions for least cost, as well as investment decisions for components marked as extendable (more on this next time).

The `n.optimize()` function can take many arguments. The most relevant for the moment is the choice of the solver. We already know some solvers from the introduction to `linopy` (e.g. "highs" and "gurobi"). They need to be installed in your environment (e.g. `conda`), to use them here:

In [ ]:
n.optimize(solver_name="highs", log_to_console=False)

Let's have a look at the results.

Since the power flow and dispatch are generally time-varying quantities, these are stored in a different location than e.g. `n.generators`. They are stored in `n.generators_t`. Thus, to find out the dispatch of the generators, run

In [ ]:
n.generators_t.p

or if you prefer it in relation to the generators nominal capacity

In [ ]:
n.generators_t.p / n.generators.p_nom

You see that the time index has the value 'now'. This is the default index when no time series data has been specified and the network only covers a single state (e.g. a particular hour). 

Similarly you will find the power flow in transmission lines at

In [ ]:
n.lines_t.p0

In [ ]:
n.lines_t.p1

The `p0` will tell you the flow from `bus0` to `bus1`. `p1` will tell you the flow from `bus1` to `bus0`.

What about the shadow prices?

In [ ]:
n.buses_t.marginal_price

## Explore the optimisation model

Under the hood, a call to `n.optimize()` builds a `linopy` optimisation model, solves it with the specified solver, and then retrieves the results back into the PyPSA data structure. We can access this intermediate `linopy` model via `n.model`. This allows us to explore the optimisation model in more detail, for instance to see all variables and constraints that were added to the optimisation problem.

In [ ]:
n.model

In [ ]:
n.model.constraints["Generator-fix-p-upper"]

In [ ]:
n.model.constraints["Bus-nodal_balance"]

In [ ]:
n.model.objective

In [ ]:
n.model.constraints["Generator-fix-p-upper"].dual.to_dataframe()

## Basic network plotting

The results can also be visualised with PyPSA's built-in plotting functions. The `n.plot()` function uses `matplotlib` to create static plots, while `n.explore()` creates interactive plots based on `pydeck`. In the following, we will focus on the interactive plotting with `n.explore()`.

The `n.explore()` function has a variety of styling arguments to tweak the appearance of the buses, the lines and the map in the background. For example, we can size the buses according to their load:

In [ ]:
s = n.loads.groupby("bus").p_set.sum()
s

In [ ]:
n.explore(
    bus_size=s,
    bus_color="orange",
    bus_alpha=1,
    line_color="orchid",
    line_width=30,
)

The `bus_size` argument of `n.explore()` can be even more powerful. 

The dispatch of each generator, we can find at:

In [ ]:
n.generators_t.p.loc["now"]

If we group this by the **bus** and **carrier** and use it as `bus_size`, we can create pie charts at each bus showing the generation mix.

In [ ]:
s = n.generators_t.p.loc["now"].groupby([n.generators.bus, n.generators.carrier]).sum()
s

Additionally, we can show the line flows by specifying the `line_flow` argument.

In [ ]:
n.explore(
    bus_size=s,
    line_width=30,
    line_flow=n.lines_t.p0.loc["now"] / 5,
)

In the backend, the plotting function will look up the colors specified in `n.carriers` for each carrier and match it with the second index-level of `s`.

## Modifying networks

Modifying data of components in an existing PyPSA network is as easy as modifying the entries of a `pandas.DataFrame`. For instance, if we want to reduce the cross-border transmission capacity between South Africa and Mozambique, we'd run:

In [ ]:
n.lines.loc["SA-MZ", "s_nom"] = 400
n.lines

In [ ]:
n.optimize(log_to_console=False)

You can see that the production of the hydro power plant was reduced and that of the gas power plant increased owing to the reduced transmission capacity.

In [ ]:
n.generators_t.p

## Global constraints for emission limits

In the example above, we happen to have some spare gas capacity with lower carbon intensity than the coal and oil generators. We could use this to lower the emissions of the system, but it will be more expensive. We can implement the limit of carbon dioxide emissions as a constraint.

This is achieved in PyPSA through **Global Constraints** which add constraints that apply to many components at once.

But first, we need to calculate the current level of emissions to set a sensible limit.

We can compute the emissions per generator (in tonnes of CO$_2$) in the following way.

$$\frac{g_{i,s,t} \cdot \rho_{i,s}}{\eta_{i,s}}$$

where $ \rho$ is the specific emissions (tonnes/MWh thermal) and $\eta$ is the conversion efficiency (MWh electric / MWh thermal) of the generator with dispatch $g$ (MWh electric):

In [ ]:
e = (
    n.generators_t.p
    / n.generators.efficiency
    * n.generators.carrier.map(n.carriers.co2_emissions)
)
e

Summed up, we get total emissions in tonnes:

In [ ]:
e.sum().sum()

So, let's say we want to reduce emissions by 10%:

In [ ]:
n.add(
    "GlobalConstraint",
    "emission_limit",
    carrier_attribute="co2_emissions",
    sense="<=",
    constant=e.sum().sum() * 0.9,
)

In [ ]:
n.optimize(log_to_console=False)

In [ ]:
n.generators_t.p

In [ ]:
n.generators_t.p / n.generators.p_nom

In [ ]:
n.global_constraints.mu

Can we lower emissions even further? Say by another 5% points?

In [ ]:
n.global_constraints.loc["emission_limit", "constant"] = 0.85

In [ ]:
n.optimize(log_to_console=False)

**No!** Without any additional capacities, we have exhausted our options to reduce emissions in that hour. The solver tells us that the problem is *infeasible*, i.e. there is no solution that satisfies all constraints. Better revert this change:

In [ ]:
n.global_constraints.loc["emission_limit", "constant"] = 0.9

## Data import and export

:::{note}
Documentation: https://docs.pypsa.org/latest/user-guide/import-export/.
:::

You may find yourself in a need to store PyPSA networks for later use. Or, maybe you want to import the genius PyPSA example that someone else uploaded to the web to explore.

Among other file formats, PyPSA can be stored as `netCDF` (`.nc`) file or as an Excel file (`.xlsx`). The approach is similar for all formats:

In [ ]:
n.export_to_netcdf("tmp.nc");

In [ ]:
n_nc = pypsa.Network("tmp.nc")

## A slightly more realistic example

**Dispatch problem with German SciGRID network**


[SciGRID](https://www.dlr.de/en/ve/research-and-transfer/projects/project-scigrid) is a project that provides an open reference model of the European transmission network. The network comprises time series for loads and the availability of renewable generation at an hourly resolution for January 1, 2011 as well as approximate generation capacities in 2014. This dataset is a little out of date and only intended to demonstrate the capabilities of PyPSA.

In [ ]:
n2 = pypsa.examples.scigrid_de()

There are some infeasibilities without allowing extension. Also, to approximate so-called $N-1$ security, we don't allow any line to be loaded above 70% of their thermal rating. $N-1$ security is a constraint that states that no single transmission line may be overloaded by the failure of another transmission line (e.g. through a tripped connection).

In [ ]:
n2.lines.s_max_pu = 0.7
n2.lines.loc[["316", "527", "602"], "s_nom"] = 1715

Because this network includes time-varying data, now is the time to look at another attribute of `n`: `n.snapshots`. Snapshots is the PyPSA terminology for time steps. In most cases, they represent a particular hour. They can be a `pandas.DatetimeIndex` or any other list-like attributes.

In [ ]:
n2.snapshots[:4]

This index will match with any time-varying attributes of components:

In [ ]:
n2.loads_t.p_set.iloc[:3, :3]

We can use simple `pandas` syntax, to create an overview of the load time series...

In [ ]:
n2.loads_t.p_set.sum(axis=1).plot(ylim=[0, 60e3], ylabel="MW")

... and the capacity factor time series:

In [ ]:
n2.generators_t.p_max_pu.T.groupby(n2.generators.carrier).mean().T.plot(ylabel="p.u.")

We can also inspect the total power plant capacities per technology...

In [ ]:
n2.generators.groupby("carrier").p_nom.sum().div(1e3).sort_values().plot.barh(
    xlabel="GW"
)


... and plot the regional distribution of loads...

In [ ]:
load = n2.loads_t.p_set.sum(axis=0).groupby(n2.loads.bus).sum()
load.head(3)

In [ ]:
n2.explore(bus_size=load / 20)

... and power plant capacities...

In [ ]:
capacities = n2.generators.groupby(["bus", "carrier"]).p_nom.sum()
capacities.head(3)

... for which we need to assign some colors to the technologies first:

In [ ]:
colors = {
    'Gas': "tomato",
    'Hard Coal': "dimgrey",
    'Run of River': "turquoise",
    'Waste': "olive",
    'Brown Coal': "peru",
    'Oil': "black",
    'Storage Hydro': "teal",
    'Other': "whitesmoke",
    'Multiple': "whitesmoke",
    'Nuclear': "deeppink",
    'Geothermal': "darkorange",
    'Wind Offshore': "lightskyblue",
    'Wind Onshore': "royalblue",
    'Solar': "gold",
    'Pumped Hydro': "lightseagreen",
    "AC": "crimson",
}
n2.add("Carrier", colors.keys(), color=colors.values(), overwrite=True)

In [ ]:
n2.explore(bus_size=capacities / 3)

So let's solve the electricity market simulation for January 1, 2011:

In [ ]:
n2.optimize(log_to_console=False)

Now, we can also plot model outputs, like the calculated power flows on the network map.

In [ ]:
line_loading = n2.lines_t.p0.iloc[0].abs() / n2.lines.s_nom / n2.lines.s_max_pu * 100  # %

In [ ]:
n2.explore(
    bus_size=1e1,
    line_color=line_loading.abs(),
    line_flow=n2.lines_t.p0.iloc[0] / 100,
    line_cmap="plasma",
    line_width=n2.lines.s_nom / 1000,
)

Or plot the hourly dispatch using PyPSA's built-in statistics functionality:

In [ ]:
n2.statistics.energy_balance.iplot()

## Statistics

The PyPSA `n.statistics` module provides a variety of pre-defined tables and plots to analyse optimisation results at different aggregation levels. For instance, the energy balance, the operational costs (OPEX), curtailment, prices and market value. We will use this module in more detail in later workshops on sector coupling and investment optimisation.

Energy balance:

In [ ]:
n2.statistics.energy_balance().div(1e3).round(1).sort_values()

Operational costs (OPEX):

In [ ]:
n2.statistics.opex().round(1).sort_values(ascending=False)

There is much more to explore in PyPSA. If you are hooked, have a look at the [documentation](https://docs.pypsa.org/) and the [examples section](https://docs.pypsa.org/latest/examples/examples/).

## Exercises

Modify some of the input data of the South Africa - Mozambique network `n`, first removing the emission limit global constraint:

In [ ]:
n.remove("GlobalConstraint", "emission_limit")

**Task 1:** Model an outage of the transmission line by removing it. How does the model compensate for the lack of transmission? 

In [ ]:
n.model.solver_model = None
n_t1 = n.copy()

n_t1.remove("Line", "SA-MZ")
n_t1.optimize(log_to_console=False)

In [ ]:
n_t1.statistics.energy_balance().astype(int).sort_values()

**Task 2:** Double the wind capacity. How is the electricity price in South Africa affected? What generator is price-setting?

In [ ]:
n.model.solver_model = None
n_t2 = n.copy()

n_t2.generators.loc["SA wind", "p_nom"] *= 2
n_t2.optimize(log_to_console=False)

In [ ]:
price = n_t2.buses_t.marginal_price
price

In [ ]:
n_t2.generators.loc[
    n.generators.marginal_cost.round(2) == price.at["now", "SA"].round(2)
].index[0]

**Task 3:** South Africa brings a new 1000 MW nuclear power plant into operation with an estimated marginal electricity generation cost of 20 €/MWh. Will that affect the electricity price?

In [ ]:
n.model.solver_model = None
n_t3 = n.copy()

n_t3.add(
    "Generator",
    "SA nuclear",
    bus="SA",
    carrier="nuclear",
    p_nom=1000,
    marginal_cost=20,
)
n_t3.optimize(log_to_console=False)

In [ ]:
n_t3.buses_t.marginal_price